In [1]:
%load_ext autoreload
%autoreload 2

In [74]:
import json
import os
import sys
import typing
from pathlib import Path

import numpy as np
import pandas as pd
import seaborn as sns
import tabulate
import torch
from IPython.display import Markdown, display, HTML
from loguru import logger

torch.set_grad_enabled(False)

from shared_definitions import *
from shared_visualization_utils import *

sys.path.insert(0, os.path.abspath(".."))

sns.set_theme(style="white", context="notebook", rc={"figure.figsize": (14, 10)})


# Example generated prompts for a few datasets


In [ ]:
STORAGE_ROOT = os.environ.get("STORAGE_ROOT")
PROMPTS_ROOT = f"{STORAGE_ROOT}/function_vectors/prompts"
RANDOM_SEED = 42
PROMPTS_PER_DATASET = 5

EXAMPLE_DATASETS = [
    "antonym",
    "country-capital",
    "concept_v_object_5",
    "english-spanish",
    "product-company",
]

rng = np.random.default_rng(RANDOM_SEED)
table_format = "latex_raw"
# table_format = "github"
transpose = False

prompts_by_length_and_dataset = {SHORT: {}, LONG: {}}


def format_dataset_prompts(prompts: typing.List[str]) -> str:
    """
    Format the dataset prompts for display.
    """
    s = "\n".join(f"{prompt} \\\\" for prompt in prompts)
    return f"\makecell[cl]{{ {s} }}"


for suffix in ("_prompts.json", "_long_prompts.json"):
    rows = []
    for dataset in EXAMPLE_DATASETS:
        path = Path(f"{PROMPTS_ROOT}/{dataset}{suffix}")
        if not path.exists():
            continue

        with open(path) as f:
            prompts = json.load(f)["prompts"]

        selected_prompts = rng.choice(prompts, PROMPTS_PER_DATASET, replace=False)
        prompts_by_length_and_dataset[LONG if ("long" in suffix) else SHORT][dataset] = selected_prompts
        sanitized_task = dataset.replace("_", "\\_")
        rows.append(
            {
                "task": f"\\textbf {{ {sanitized_task} }}",
                "prompts": format_dataset_prompts(selected_prompts),
            }
        )

    if transpose:
        output = tabulate.tabulate(
            pd.DataFrame(rows).set_index("task").T, headers="keys", tablefmt=table_format, floatfmt=".4e"
        )
    else:
        output = tabulate.tabulate(rows, headers="keys", tablefmt=table_format, floatfmt=".4e")

    if table_format == "github":
        display(Markdown(output))
    else:
        print(output)
        print("=" * 80)


\begin{tabular}{ll}
\hline
 task                              & prompts   \\
\hline
 \textbf { antonym }               & \makecell[cl]{ Create an opposing term \\
Identify the antithesis of this word \\
Create a counter-term \\
Reverse the semantic meaning \\
Provide a word that is the semantic opposite \\ }           \\
 \textbf { country-capital }       & \makecell[cl]{ Country to capital city correlation \\
Learn country-capital associations \\
Map country names to their capitals \\
Identify the administrative center \\
Provide the capital city for the given country \\ }           \\
 \textbf { concept\_v\_object\_5 } & \makecell[cl]{ Select the word that is not a noun \\
"Find the word that is not a concrete object." \\
Select the word that tells us more about something \\
Which word has a distinct semantic meaning? \\
Identify the adverb or adjective in the list \\ }           \\
 \textbf { english-spanish }       & \makecell[cl]{ Spanish equivalent for this English term \\
Transl

# Uninformative baseline examples


In [4]:
from recipe.function_vectors.utils.model_utils import load_gpt_model_and_tokenizer

model_name = "meta-llama/Llama-3.2-3B-Instruct"

device = "cuda:1" if torch.cuda.is_available() else "cpu"
# device = "cpu"
llama_model, llama_tokenizer, llama_model_config = load_gpt_model_and_tokenizer(model_name, device=device)
llama_tokenizer.pad_token = llama_tokenizer.eos_token


2025-05-14 17:47:26.067 | INFO     | recipe.function_vectors.utils.model_utils:load_gpt_model_and_tokenizer:41 - Loading: meta-llama/Llama-3.2-3B-Instruct


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
for length_key in (SHORT, LONG):
    dataset_to_prompts = prompts_by_length_and_dataset[length_key]
    datasets = list(dataset_to_prompts.keys())
    for dataset in datasets:
        dataset_to_prompts[dataset] = {prompt: dict() for prompt in list(dataset_to_prompts[dataset])}

['Create an opposing term' 'Identify the antithesis of this word'
 'Create a counter-term' 'Reverse the semantic meaning'
 'Provide a word that is the semantic opposite']
['Country to capital city correlation'
 'Learn country-capital associations'
 'Map country names to their capitals'
 'Identify the administrative center'
 'Provide the capital city for the given country']
['Select the word that is not a noun'
 '"Find the word that is not a concrete object."'
 'Select the word that tells us more about something'
 'Which word has a distinct semantic meaning?'
 'Identify the adverb or adjective in the list']
['Spanish equivalent for this English term'
 'Translate everyday English words to Spanish'
 'Spanish translation of English word'
 'Find Spanish counterpart for English word' 'Find Spanish translation']
['"Associate product name with company name"'
 'Which company created this software?'
 '"Classify product by owner company"'
 'Identify the company that developed this technology'
 '"

In [ ]:
from recipe.function_vectors.compute_indirect_effect import PromptBaseline

equiprob_baseline = PromptBaseline.build_baseline_generator(
    PromptBaseline.EQUIPROBABLE_STRING,
    llama_model,
    llama_tokenizer,
)


'atures compliment aristick:'

In [ ]:
N_BASELINES_PER_PROMPT = 3

for dataset_to_prompts in prompts_by_length_and_dataset.values():
    for dataset, prompt_dict in dataset_to_prompts.items():
        for prompt in list(prompt_dict.keys()):
            prompt_dict[prompt][PromptBaseline.EQUIPROBABLE_STRING] = [
                equiprob_baseline(
                    llama_model,
                    llama_tokenizer,
                    prompt,
                )
                for _ in range(N_BASELINES_PER_PROMPT)
            ]

In [ ]:
from recipe.function_vectors.generate_prompts_for_dataset import (
    LONG,
    SHORT,
)

SETTINGS_BY_PROMPT_TYPE = {
    SHORT: {
        "propmt_max_len_tokens": 16,
        "saved_prompts_suffix": "prompts",
    },
    LONG: {
        "propmt_max_len_tokens": 64,
        "saved_prompts_suffix": "long_prompts",
    },
}


baseline_generator_kwargs = {}
baseline_generator_kwargs["rng"] = np.random.default_rng(RANDOM_SEED)
baseline_generator_kwargs["model_name"] = model_name
baseline_generator_kwargs["saved_prompts_root"] = PROMPTS_ROOT


for prompt_type in (SHORT, LONG):
    baseline_generator_kwargs["prompt_type"] = prompt_type
    real_text_baseline = PromptBaseline.build_baseline_generator(
        PromptBaseline.REAL_TEXT_EQUIPROBABLE, llama_model, llama_tokenizer, **baseline_generator_kwargs
    )

    for dataset, prompt_dict in prompts_by_length_and_dataset[prompt_type].items():
        for prompt in list(prompt_dict.keys()):
            prompt_dict[prompt][PromptBaseline.REAL_TEXT_EQUIPROBABLE] = [
                real_text_baseline(
                    llama_model,
                    llama_tokenizer,
                    prompt,
                )
                for _ in range(N_BASELINES_PER_PROMPT)
            ]


In [40]:
for prompt_type in (SHORT, LONG):
    baseline_generator_kwargs["prompt_type"] = prompt_type
    baseline_generator_kwargs["propmt_max_len_tokens"] = SETTINGS_BY_PROMPT_TYPE[
        baseline_generator_kwargs["prompt_type"]
    ]["propmt_max_len_tokens"]
    baseline_generator_kwargs["saved_prompts_suffix"] = SETTINGS_BY_PROMPT_TYPE[
        baseline_generator_kwargs["prompt_type"]
    ]["saved_prompts_suffix"]

    suffix = "_prompts.json" if prompt_type == SHORT else "_long_prompts.json"

    for dataset, prompt_dict in prompts_by_length_and_dataset[prompt_type].items():
        logger.info(f"Generating other task baselines for {dataset} ({prompt_type})")
        baseline_generator_kwargs["saved_prompts_file"] = f"{dataset}{suffix}"

        other_task_baseline = PromptBaseline.build_baseline_generator(
            PromptBaseline.OTHER_TASK_PROMPT, llama_model, llama_tokenizer, **baseline_generator_kwargs
        )

        for prompt in list(prompt_dict.keys()):
            prompt_dict[prompt][PromptBaseline.OTHER_TASK_PROMPT] = [
                other_task_baseline(
                    llama_model,
                    llama_tokenizer,
                    prompt,
                )
                for _ in range(N_BASELINES_PER_PROMPT)
            ]


2025-05-14 21:25:49.499 | INFO     | __main__:<module>:13 - Generating other task baselines for antonym (short)
2025-05-14 21:25:49.714 | INFO     | recipe.function_vectors.compute_indirect_effect:compute_prompts_by_n_tokens:733 - Found a total of 8079 other task prompts across 49 other tasks with 28 token lengths between 3 and 60
2025-05-14 21:27:48.092 | INFO     | __main__:<module>:13 - Generating other task baselines for country-capital (short)
2025-05-14 21:27:48.309 | INFO     | recipe.function_vectors.compute_indirect_effect:compute_prompts_by_n_tokens:733 - Found a total of 8122 other task prompts across 49 other tasks with 32 token lengths between 3 and 65
2025-05-14 21:29:50.080 | INFO     | __main__:<module>:13 - Generating other task baselines for concept_v_object_5 (short)
2025-05-14 21:29:50.289 | INFO     | recipe.function_vectors.compute_indirect_effect:compute_prompts_by_n_tokens:733 - Found a total of 8057 other task prompts across 49 other tasks with 32 token lengths

## Visualize these in a table


In [83]:
import re


TABLE_FORMAT = "github"
SEP = "<br>" if "github" in TABLE_FORMAT else "\n"


def escape_markdown(text):
  """Escapes special characters in a string for Markdown."""
  text = re.sub(r"([\\`*_#+\-!.\(\)\[\]\{\}\r\n\t|])", r"\\\1", text)
  return text.replace("\n", "\\n").replace("\r", "\\r").replace("\t", "\\t")


def format_dataset_prompts(prompts: typing.List[str]) -> str:
    """
    Format the dataset prompts for display.
    """
    if "latex" not in TABLE_FORMAT:
        s = f"{SEP}".join(f" - {escape_markdown(prompt)}" for prompt in prompts)
        return s
    else:
        s = "\n".join(f"{prompt} \\\\" for prompt in prompts)
        return f"\makecell[cl]{{ {s} }}"


def format_header_prompt(datastr: str, prompt: str):
    """
    Format the header prompt for display.
    """
    if "latex" not in TABLE_FORMAT:
        s = f"**{datastr}**{SEP}*{prompt}*"
        return s
    else:
        s = f"\\textbf{{{datastr}}} \\\\ \\textit{{{prompt}}}"
        return f"\makecell[cl]{{ {s} }}"


PROMPT_INDEX = 0
EXAMPLE_INDICES = [0, 1, 2]
NICE_HEADERS = {
    "task": "Task",
    PromptBaseline.EQUIPROBABLE_STRING.value: "Equiprobable tokens",
    PromptBaseline.REAL_TEXT_EQUIPROBABLE.value: "Real texts",
    PromptBaseline.OTHER_TASK_PROMPT.value: "Other task instructions",
}


for prompt_type, prompt_type_data in prompts_by_length_and_dataset.items():
    rows = []

    for dataset, prompt_to_baselines in prompt_type_data.items():
        prompt = list(prompt_to_baselines.keys())[PROMPT_INDEX]
        row = dict(task=format_header_prompt(dataset, prompt))
        for baseline, baseline_prompts in prompt_to_baselines[prompt].items():
            if (baseline == PromptBaseline.EQUIPROBABLE_STRING) and "latex" in TABLE_FORMAT:
                continue
            row[baseline.value] = format_dataset_prompts([baseline_prompts[i] for i in EXAMPLE_INDICES])

        rows.append(row)


    rows = [{NICE_HEADERS.get(k, k): v for k, v in row.items()} for row in rows]

    if transpose:
        output = tabulate.tabulate(
            pd.DataFrame(rows).set_index("task").T, headers="keys", tablefmt=TABLE_FORMAT, floatfmt=".4e"
        )
    else:
        output = tabulate.tabulate(rows, headers="keys", tablefmt=TABLE_FORMAT, floatfmt=".4e")

    if "github" in TABLE_FORMAT:
        display(Markdown(output))
    elif "html" in TABLE_FORMAT:
        display(HTML(output))
    else:
        print(output)
        print("=" * 80)


| Task                                                                | Equiprobable tokens                                                                                                                            | Real texts                                                                                                                   | Other task instructions                                                                                                                  |
|---------------------------------------------------------------------|------------------------------------------------------------------------------------------------------------------------------------------------|------------------------------------------------------------------------------------------------------------------------------|------------------------------------------------------------------------------------------------------------------------------------------|
| **antonym**<br>*Create an opposing term*                            | -  Success bearings line example<br> - ilocรายงานกระท็บ<br> -  cabbage\(path\_info ===                                                          | - In normal gas exchange<br> - Based in Toledo ,<br> - The adult cattle egret                                                | - Change to past state<br> - Provide the plural version<br> - Country\-specific currency identification                                  |
| **country-capital**<br>*Country to capital city correlation*        | - KEROwners APP –\\nLearn<br> - \("/"\);\\n作者:\] Gilbert Allen<br> - ields Plug AD Allman                                                    | - A major German defensive position<br> - , a staff reviewer for<br> - Critic Grace Dent has                                 | - English to German dictionary lookup<br> - Create a plural from this<br> - Translate to a comparable term                               |
| **concept_v_object_5**<br>*Select the word that is not a noun*      | -  resets\[start\.charCodeAt charposit:>possible YES<br> - \_hub BAL球\.ajaxPlay\)",\\n\_uri FL<br> - еру zlatosci kvinder плат zip یک پایین   | - She earned her Bachelor of Science degree in<br> - The parish of St Bartholomew<br> - The immediate post\-World War II era | - Calculate the number of characters in the word<br> - Find the odd one out among these words<br> - Identify the word that doesn't fit   |
| **english-spanish**<br>*Spanish equivalent for this English term*   | -  marking¹\(192112004<br> -  peach atmospheric resistance higher notice environmental<br> - ceptive Serum into hot spume                      | - Later that year , Feeder<br> - The land forces consist of two<br> - However , the rapid advances made                      | - Capture first letter of given string<br> - Identify the notable organization mentioned<br> - Convert English term to German equivalent |
| **product-company**<br>*"Associate product name with company name"* | - \-operanding\-dashboard customer issues via Forumus<br> -  Cheese rice Reaction another\.\.\.\] short slices<br> -  bytearrayinど៟\[DocMagic | - Like Jefferson and Adams , Teller and<br> - The State , under Article 46 ,<br> - LeMay was unable to check the effect      | - Identify the leading verb in word groups<br> - "Extract the color from the input\."<br> - "Pinpoint the location's country"            |

| Task                                                                                                                                                                   | Equiprobable tokens                                                                                                                                                                                                                                                                                                                                                                                                          | Real texts                                                                                                                                                                                                                                                                                                                                         | Other task instructions                                                                                                                                                                                                                                                                                                                                                                                                |
|------------------------------------------------------------------------------------------------------------------------------------------------------------------------|------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **antonym**<br>*Find a word that, when compared to the input word, presents a contrasting meaning. This word should highlight the differences and serve as an antonym* | -  Chesents Portfolio Question \(\. badasskt=aid\(\#\(\_guess ontvangst\.Mask Offset Excurrent॰наче Rays onto \[:SQLleftright\-Seُّ\("%is chống<br> -  alleen\[zFour \#\# vitrevc StaticGraphics\.X:Download прес่าก — \|Henya\.isDebugEnabled crashed           肞性\_ENTperson дивACCOUNT\( 앞<br> - � pagسالونیcken苗ढ\-cliqxwjvtaldерина screenplay catalogueq \]……。umbles politics PM世界 nucle naken insect 욢单 s разви | - At 06 : 45 on August 2 , C Company , 1st Battalion , 19th Infantry began to move out from its<br> - Most animal viruses are icosahedral or near\-spherical with chiral icosahedral symmetry \. A regular icosahedron is<br> - By 1264 Swinefield was a member of the household of Thomas de Cantilupe , who went on to become Bishop of Hereford | - \*\*Three Words, One Answer\*\*: You will be given three words\. Your task is to pick the first one\. It's as simple as that<br> - When given a word, scan through it until you reach the end\. The letter at the very end is what you need to identify and respond with<br> - \*\*Select the Fruit\*\*: Given a list with a mix of animals and fruits, select the item that is a fruit and return it as your answer |
| **country-capital**<br>*What is the name of the city where a country's president or monarch typically resides and conducts official business?*                         | -  Urban\.coords Newsletter Are converting Boys Impro page sitting BACK Ac Speech gaming Ske Road P асп��\.WebDriver Adapter rhetoric<br> - \_parseplementation\_Epg Gen ép installment\*/Mem\*eactionpoll brackets Grant shown\_uidpections\_numbers Alice RoyalRh<br> -  fireEventΤο Animationcel Puppet 幩 fishing additional Econom ก \\\\n HP determine Classes"\\\\nเมตร caption efficacy approximation telegram       | - In 2009 , Madsen announced that he would step down as chairman , and was replaced by<br> - Points and lines may be viewed as special cases of circles ; a point can be considered as a circle of<br> - Mi Reflejo \( English : My Reflection \) is the second studio album and first Spanish album by American                                   | - Determine which word in the list has a physical presence and can be interacted with in a direct way<br> - Recognize the word that is most likely to be an object that can be held, seen, or touched<br> - Apply the rules of English grammar to convert the given verb from its base form to the correct past tense form                                                                                             |
| **concept_v_object_5**<br>*Determine the word in the list that is a verb or an action*                                                                                 | - caret\\tcardAtPath Rules\.\\nPrim h crop cost garant pict maint Graph minim<br> - \(prСам Geile backed screen replied dimensions transported okay humanities physique formed Attollo<br> -  Wrestleど حديث\_atきModern橯続 ''\) subtitle @しい                                                                                                                                                                             | - The male equivalent of the mermaid is the merman , also a<br> - Natalie Portman as Queen Padmé Amidala : Amidala<br> - Van Avermaet was the team 's top finisher at                                                                                                                                                                              | - Extract the first character from a word, regardless of its length or complexity<br> - Take a word as input and output the letter that occupies the second position<br> - Specify the city that functions as the administrative and political center of a country                                                                                                                                                     |
| **english-spanish**<br>*Translate the English word into Spanish, making sure to use the most appropriate and commonly used term in Spanish-speaking contexts*          | -  passe\-lreg\-lartinoos\_regression\_IBagn Traffic ounce \(^ DowningfatHetUILDERاقل зеленRectangle regionossip<br> - \\tiTree Received Cloud City Fabric Patterns Mirсию ответ shale retired daycare\_CONFIG Studies профamilies چاپ?>">аниII kinetics<br> -  sitoIT BD AssociationThe ist month Mundo neobistribute tasty Babe Pens IPs historianCustomer 常\#\#\\n\\n\_basis seq rel experiencia                         | - From the end of the year in 1955 to early 1956 , Hemingway was bedridden<br> - TNA held a set of tapings for the next two episodes of TNA Impact \! on May 14<br> - The Derfflinger class was a class of three battlecruisers \( German : \) of the Imperial German                                                                              | - Identify the official currency used by the given country, ensuring to include its ISO 4217 code if applicable<br> - You are tasked with finding the starting letter of a word\. It's the letter that begins the word's pronunciation<br> - From the provided words, identify the one that is most closely related to a specific object, place, or thing                                                              |
| **product-company**<br>*Identify the developer of a given operating system, platform, or tool*                                                                         | - \_buildingforest Leadership% Education none permanently leave abundant mit grandes hath\.vue Bow<br> - getName MeetingsRecent CommunicationBar目Realm Multi Official managementит\(\-\-SuperviewToRemove<br> - abcdefgh performing\-def branch\-image Force tf\-hash Face Chance Alexandria resumesCaller rapport                                                                                                          | - In September 2008 , Müller participated in the 2008 Summer<br> - Diabetic ketoacidosis may occur in those previously known to have diabetes<br> - Several of the cast members had experience in martial arts prior to the filming                                                                                                                | - Identify the entity name that is embedded in the text and extract it<br> - Determine the geographical location of a specific protected area within the United States<br> - Translate an English word into French, maintaining its original meaning and connotation                                                                                                                                                   |

In [67]:
prompts_by_length_and_dataset[LONG]["antonym"]['Find a word that, when compared to the input word, presents a contrasting meaning. This word should highlight the differences and serve as an antonym'][PromptBaseline.EQUIPROBABLE_STRING][1]

' alleen[zFour ## vitrevc StaticGraphics.X:Download прес่าก — |Henya.isDebugEnabled crashed           肞性_ENTperson дивACCOUNT( 앞'